# Setup - 
Run jupyter-notebook in env and pick link


In [1]:
import logging
# show both debug and warning messages
logging.basicConfig(level=logging.DEBUG)

In [2]:
import sys
sys.path.insert(0, "/NS/llm-pretraining/work/afkhan/tokensmith")

In [3]:
import sys
sys.path.insert(0, "/NS/llm-pretraining/work/afkhan/USC_Colab/gpt-neox")

In [4]:
from transformers import AutoTokenizer
TOKENIZER_NAME_OR_PATH = "EleutherAI/gpt-neox-20b"
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME_OR_PATH, add_eos_token=True)

/NS/venvs/work/afkhan/neox_updated_env/lib/python3.11/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /EleutherAI/gpt-neox-20b/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /api/resolve-cache/models/EleutherAI/gpt-neox-20b/c292233c833e336628618a88a648727eb3dff0a7/tokenizer_config.json HTTP/1.1" 200 0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# import os
# import torch
# import torch.distributed as dist

# def setup():
#     # Set required env vars manually
#     os.environ['RANK'] = '0'
#     os.environ['WORLD_SIZE'] = '1'
#     os.environ['MASTER_ADDR'] = '127.0.0.1'
#     os.environ['MASTER_PORT'] = '29500'

#     dist.init_process_group(backend='nccl', init_method='env://')

# setup()

# import torch
# import os

# os.environ['MASTER_ADDR'] = '127.0.0.1'
# os.environ['MASTER_PORT'] = '29500'
# torch.distributed.init_process_group(backend='nccl', rank=0, world_size=1)

In [6]:
from tokensmith.manager import DatasetManager

[2025-06-30 16:51:43,642] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/NS/venvs/work/afkhan/neox_updated_env/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/NS/venvs/work/afkhan/neox_updated_env/lib/python3.11/site-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/NS/llm-pretraining/work/afkhan/tokensmith/artifacts, stdin=None, shell=False, universal_newlines=False)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/NS/llm-pretraining/work/afkhan/tokensmith/artifacts, stdin=None, shell=False, universal_newlines=False)
DEBUG:wandb.docker.auth:Trying paths: ['/home/afkhan/.docker/config.json', '/home/afkhan/.dockercfg']
DEBUG:wandb.docker.auth:Found file at path: /home/afkhan/.docker/c

In [7]:
dataset_manager = DatasetManager()

# Setup Search

In [8]:
# dataset_manager.setup_search(
#     bin_file_path="/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.bin",
#     search_index_save_path="/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/search_index_text_document.idx",
#     vocab=2**32,
#     verbose=True,
#     reuse=True,
# )

In [9]:
dataset_manager.setup_edit_inspect_sample_export(
    dataset_prefix='/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document',
    # batch_info_prefix='dataset_train_indexmap_100ns_2048sl_1s_packedpi_ac',
    batch_info_save_prefix='/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/batch_info',
    train_iters=100,
    train_batch_size=16,
    train_seq_len=2048,
    seed=42,
    splits_string='990,5,5',
    packing_impl='packed',
    allow_chopped=True,
)

DEBUG:tokensmith.utils:Initializing WriteableMMapIndexedDataset with pointer: /NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.bin and index: /NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.idx
DEBUG:tokensmith.utils:Loading doc/sample/shuffle indexes with prefix: /NS/llm-pretraining/work/afkhan/tokensmith/artifacts/batch_info


    warming up index mmap file...
    reading sizes...
    reading pointers...
    reading document index...


In [10]:
resp = dataset_manager.sample.sample_by_indexes([10], return_doc_details=True)

In [12]:
resp = dataset_manager.inspect.inspect_sample_by_id(10, return_doc_details=True, return_detokenized = True, tokenizer=tokenizer)

In [11]:
from tokensmith.utils import generate_training_sample

In [12]:
generate_training_sample(resp[0][0], tokenizer)

' day, they find a big club on the grass. It is brown and heavy. "Look, a club!" Lily says. "Let\'s play with it!" "OK!" Ben says. "What can we do with it?" Lily thinks for a moment. Then she has an idea. "We can spin it!" she says. "Like this!" She holds the club with both hands and turns around fast. The club makes a whoosh sound in the air. "Wow, that looks fun!" Ben says. "Can I try?" "Sure!" Lily says. She gives the club to Ben. Ben tries to spin it, but it is too heavy for him. He falls down and drops the club. "Ouch!" Ben says. "That hurts!" Lily laughs. She helps Ben get up. "You are lazy!" she says. "You need to be strong to spin the club!" Ben makes a face. "I am not lazy!" he says. "I am just small. You are big and strong. That\'s not fair!" Lily smiles. She hugs Ben. "Don\'t be sad," she says. "We can play something else. How about we swing on the swings?" "OK!" Ben says. "That sounds fun!" They leave the club on the grass and run to the swings. They swing high and low, and

In [10]:
import numpy as np

RNG = np.random.default_rng(42)

In [12]:
dataset_manager.edit.inject_and_preview(
    text = 'This is a test sentence.',
    tokenizer=tokenizer,
    injection_loc=699,
    injection_type="seq_shuffle",
    rng=RNG,
    add_eos_token=True,
    dry_run=False,
)

Dummy sample: [1552  310  247 1071 6197   15    0]
Training sample 699
Sample consists of segments from 12 documents
Raw sample: [16522   497   594 ...  5972    15   380]
---
Decoded sample:  Dad were so excited to show me their new thing. It was tall and wide with a shapely top. It was bright red and had pretty decorations. I was so amazed! Mum and Dad had designed a beautiful red chair for me. I squealed with delight as I hugged them and spun around in it. It was the best thing ever!<|endoftext|>Once upon a time, there was a little bird with gray feathers. The bird loved to fly high in the sky and sit on a branch. One day, the bird saw a big, juicy worm on the ground. The bird wanted to eat the worm but his mother told him it was not safe to eat worms on the ground. She said only worms on the branch were safe to eat. But the little bird didn't listen to his mother and flew down to the ground to eat the worm. As he was eating, a big, mean cat came and chased after him. The little bird

DEBUG:tokensmith.utils:>>> Inserting 72 of 2049 tokens starting at position 8562636
DEBUG:tokensmith.utils:>>> Injection Data Shape (72,) | Injection Data Dtype uint16
DEBUG:tokensmith.utils:>>> Inserting 189 of 2049 tokens starting at position 7462992
DEBUG:tokensmith.utils:>>> Inserting 152 of 2049 tokens starting at position 4710212
DEBUG:tokensmith.utils:>>> Inserting 242 of 2049 tokens starting at position 3269832
DEBUG:tokensmith.utils:>>> Inserting 173 of 2049 tokens starting at position 1322428
DEBUG:tokensmith.utils:>>> Inserting 188 of 2049 tokens starting at position 4877468
DEBUG:tokensmith.utils:>>> Inserting 143 of 2049 tokens starting at position 3814036
DEBUG:tokensmith.utils:>>> Inserting 227 of 2049 tokens starting at position 1944972
DEBUG:tokensmith.utils:>>> Inserting 205 of 2049 tokens starting at position 8853872
DEBUG:tokensmith.utils:>>> Inserting 211 of 2049 tokens starting at position 1577198
DEBUG:tokensmith.utils:>>> Inserting 156 of 2049 tokens starting at

Training sample 699 after injection
Raw sample: [ 921  479  616 ...  285 8513 5972]
---
Decoded sample:  show me their new thing. It was tall and wide with a shapely top. It was bright red and had pretty decorations. I was so amazed! Mum and Dad had designed a beautiful red chair for me. I squealed with delight as I hugged them and spun around in it. It was the best thing ever!<|endoftext|>This is a test sentence.<|endoftext|>Once upon a time, there was a little bird with gray feathers. The bird loved to fly high in the sky and sit on a branch. One day, the bird saw a big, juicy worm on the ground. The bird wanted to eat the worm but his mother told him it was not safe to eat worms on the ground. She said only worms on the branch were safe to eat. But the little bird didn't listen to his mother and flew down to the ground to eat the worm. As he was eating, a big, mean cat came and chased after him. The little bird tried to fly away, but he was too slow. The cat caught him and ate him f

In [ ]:
tokenizer.decode([1552 , 310 , 247 ,1071 ,6197 ,  15])

'This is a test sentence.'

In [ ]:
tokenizer('This is a test sequence.', add_special_tokens=True)

{'input_ids': [1552, 310, 247, 1071, 3425, 15], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.eos_token_id

0

In [ ]:
test_sample = list(tokenizer("big")["input_ids"])

In [ ]:
test_sample

[2760]

In [ ]:
dataset_manager.search.count(test_sample)

AttributeError: 'NoneType' object has no attribute 'count'

In [ ]:
from tokengrams import MemmapIndex

# Create a new index from an on-disk corpus of u16 tokens and save it to a .idx file. 
# Set verbose to true to include a progress bar for the index sort.
index = MemmapIndex.build(
    "/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document.bin",
    "/NS/llm-pretraining/work/afkhan/tokensmith/artifacts/data_tokenized_text_document_search_2_32.idx",
    vocab=2**16,
)

# True for any valid index.
print(index.is_sorted())
  

Sorting indices...
True


In [ ]:
# Get the count of "hello world" in the corpus.
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
print(index.count(tokenizer.encode("Once")))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


15326


In [ ]:
tokenizer.encode("once")

[19131]

In [ ]:
tokenizer.encode("big")

[2760]